In [16]:
import pandas as pd
import numpy as np
import os 
import librosa           
import librosa.display   
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold 
from sklearn.preprocessing import StandardScaler, LabelEncoder 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC                   
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix 
import matplotlib.pyplot as plt 
import seaborn as sns    
import warnings
warnings.filterwarnings('ignore')       

base_path = "C:/Users/anish/OneDrive/Desktop/ML/MusicClassifier/Data/genres_original"
all_files = os.listdir(base_path)
list_of_rows  = []
for genre_folder in all_files:
    genre_path = os.path.join(base_path, genre_folder)
    songs = os.listdir(genre_path)
    for song in songs:
        try:
            song_path = os.path.join(genre_path, song)
            y, sr = librosa.load(song_path, sr=None)
            zcr = librosa.feature.zero_crossing_rate(y)
            zcr_mean = np.mean(zcr)
            zcr_sd = np.std(zcr)
            spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
            sc_mean = np.mean(spectral_centroid)
            sc_sd = np.std(spectral_centroid)
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_fft=256, n_mels=40)
            mfcc_mean = np.mean(mfcc)
            mfcc_sd = np.std(mfcc)
            new_row = {'Genre': genre_folder, 'ZCR':zcr_mean, 'Spectral Centroid':sc_mean, 'MFCC':mfcc_mean}
            list_of_rows.append(new_row)
        except:
            continue

music_analytics = pd.DataFrame(list_of_rows)


